In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50

In [ ]:
# import the necessary packages

import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter detections by (1) ensuring that the object
			# detected was a person and (2) that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)



	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	# return the list of results
	return results

In [ ]:
# USAGE
# python social_distance_detector.py --input pedestrians.mp4
# python social_distance_detector.py --input pedestrians.mp4 --output output.avi

# import the necessary packages
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input","/content/drive/MyDrive/distance/pedestrians.mp4","--output","my_output.avi","--display","1"]))

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolo-coco/coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolov3.weights"])
configPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolo-coco/yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None

# loop over the frames from the video stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# resize the frame and then detect people (and only people) in it
	frame = imutils.resize(frame, width=700)
	results = detect_people(frame, net, ln,
		personIdx=LABELS.index("person"))

	# initialize the set of indexes that violate the minimum social
	# distance
	violate = set()

	# ensure there are *at least* two people detections (required in
	# order to compute our pairwise distance maps)
	if len(results) >= 2:
		# extract all centroids from the results and compute the
		# Euclidean distances between all pairs of the centroids
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# check to see if the distance between any two
				# centroid pairs is less than the configured number
				# of pixels
				if D[i, j] < MIN_DISTANCE:
					# update our violation set with the indexes of
					# the centroid pairs
					violate.add(i)
					violate.add(j)

	# loop over the results
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# extract the bounding box and centroid coordinates, then
		# initialize the color of the annotation
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0)

		# if the index pair exists within the violation set, then
		# update the color
		if i in violate:
			color = (0, 0, 255)

		# draw (1) a bounding box around the person and (2) the
		# centroid coordinates of the person,
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# draw the total number of social distancing violations on the
	# output frame
	text = "Social Distancing Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	# check to see if the output frame should be displayed to our
	# screen
	# if args["display"] > 0:
	# 	# show the output frame
	# 	#cv2_imshow(frame)
	# 	key = cv2.waitKey(1) & 0xFF

	# 	# if the `q` key was pressed, break from the loop
	# 	if key == ord("q"):
	# 		break

	# if an output video file path has been supplied and the video
	# writer has not been initialized, do so now
	if args["output"] != "" and writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		writer.write(frame)

[INFO] loading YOLO from disk...
[INFO] accessing video stream...


In [ ]:
!pip install moviepy

In [ ]:
!pip install ffmpeg-python

In [ ]:

import ffmpeg

def convert_avi_to_mp4(input_path, output_path):
    try:
        ffmpeg.input(input_path).output(output_path).run()
        print("File converted successfully!")
    except ffmpeg.Error as e:
        print(f"Error converting file: {e.stderr}")

# Specify the paths for the input AVI file and the output MP4 file
avi_file_path = '/content/my_output.avi'
mp4_file_path = '/content/output_file.mp4'

# Convert the AVI file to MP4
convert_avi_to_mp4(avi_file_path, mp4_file_path)

File converted successfully!


In [ ]:
from IPython.display import Video

def play_mp4_file(file_path):
    # Display the video
    return Video(file_path, embed=True,width = 1080)

# Specify the path to your MP4 file
mp4_file_path = '/content/output_file.mp4'

# Call the function to play the MP4 file
play_mp4_file(mp4_file_path)

UPGRADED DISPLAYING THE RESULTS IN REAL TIME WEBSITE  USING FLASK AND NGROK

In [ ]:
!pip install flask_ngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,081 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [2,410 B]
Get:13 http://archi

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [ ]:
!pip install flask


In [ ]:
port_no = 5000

In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=9fef34953851cd1fcc3d4cb18464a1985e893592079106670d8abe0600a867fc
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
import cv2
from flask import Flask, render_template, Response,render_template_string
import threading
from base64 import b64encode
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import os

# Initialize Flask application
app = Flask(__name__)
ngrok.set_auth_token("2RulHmqco9uyS22yVfHzlgao4cc_3nLW36fxJJjj3tbrGt6Vf")
public_url =  ngrok.connect(port_no).public_url

# Construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
    help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
    help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
    help="whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input", "/content/drive/MyDrive/distance/pedestrians.mp4",
                           "--output", "my_output.avi", "--display", "1"]))

# Load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolo-coco/coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# Derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolov3.weights"])
configPath = os.path.sep.join(["/content/drive/MyDrive/distance/yolo-coco/yolov3.cfg"])

# Load our YOLO object detector trained on the COCO dataset (80 classes)
print("[INFO] Loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# Determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize the video stream and pointer to the output video file
print("[INFO] Accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None

# Initialize an empty list to store the processed frames
processed_frames = []

# Function to detect people and calculate social distancing violations
def detect_people(frame, net, ln, personIdx=0):
    (H, W) = frame.shape[:2]
    results = []

    # Construct a blob from the input frame and then perform a forward pass of the YOLO object detector
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # Initialize the lists of detected bounding boxes, centroids, and confidence
    boxes = []
    centroids = []
    confidences = []

    # Loop over the layer outputs
    for output in layerOutputs:
        # Loop over each detection
        for detection in output:
            # Extract the class ID and confidence
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Filter detections for people class and minimum confidence
            if classID == personIdx and confidence > 0.5:
                # Scale the bounding box coordinates
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # Calculate the top-left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # Update the lists
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    # Apply non-maxima suppression to suppress weak, overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

    # Ensure at least one detection exists
    if len(idxs) > 0:
        # Loop over the indexes
        for i in idxs.flatten():
            # Extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # Add the detection result to the list of results
            results.append((confidences[i], (x, y, x + w, y + h), centroids[i]))

    # Return the list of results
    return results

# Function to process the frames
def process_frames():
    global processed_frames, vs, writer

    # Loop over the frames from the video stream
    while True:
        # Read the next frame from the file
        (grabbed, frame) = vs.read()

        # If the frame was not grabbed, then we have reached the end of the stream
        if not grabbed:
            break

        # Resize the frame and then detect people (and only people) in it
        frame = imutils.resize(frame, width=700)
        results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

        # Initialize the set of indexes that violate the minimum social distance
        violate = set()

        # Ensure there are at least two people detections (required to compute pairwise distance maps)
        if len(results) >= 2:
            # Extract all centroids from the results and compute the Euclidean distances between all pairs of the centroids
            centroids = np.array([r[2] for r in results])
            D = dist.cdist(centroids, centroids, metric="euclidean")

            # Loop over the upper triangular of the distance matrix
            for i in range(0, D.shape[0]):
                for j in range(i + 1, D.shape[1]):
                    # Check if the distance between any two centroid pairs is less than the configured number of pixels
                    if D[i, j] < 50:
                        # Update the violation set with the indexes of the centroid pairs
                        violate.add(i)
                        violate.add(j)

        # Loop over the results
        for (i, (prob, bbox, centroid)) in enumerate(results):
            # Extract the bounding box and centroid coordinates
            (startX, startY, endX, endY) = bbox
            (cX, cY) = centroid
            color = (0, 255, 0)

            # If the index pair exists within the violation set, then update the color
            if i in violate:
                color = (0, 0, 255)

            # Draw a bounding box around the person and the centroid coordinates
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.circle(frame, (cX, cY), 5, color, 1)

        # Draw the total number of social distancing violations on the output frame
        text = "Social Distancing Violations: {}".format(len(violate))
        cv2.putText(frame, text, (10, frame.shape[0] - 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

        # If the video writer is not None, write the frame to the output video file
        if writer is not None:
            writer.write(frame)

        # Append the processed frame to the list
        processed_frames.append(frame)

    # Release the video writer and video stream
    if writer is not None:
        writer.release()
    vs.release()


# Route for the home page
@app.route('/')
def index():
    html = '''
    <!DOCTYPE html>
    <html>
      <head>
        <title>Social Distancing Detection</title>
        <style>
          body {
            background-image: url('https://example.com/social_distance.gif');
            background-repeat: no-repeat;
            background-size: cover;
          }
          .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
          }
          .frame {
            border: 1px solid #ccc;
            box-shadow: 0px 0px 5px 0px rgba(0,0,0,0.5);
            padding: 10px;
          }
        </style>
      </head>
      <body>
        <h1>Social Distancing Detection</h1>
        <div class="container">
          <div class="frame">
            <img src="{{ url_for('video_feed') }}" width="700" height="500" />
          </div>
        </div>
      </body>
    </html>
    '''
    return render_template_string(html)

# Function to generate video frames
def generate_frames():
    global processed_frames

    # Loop over the processed frames
    for frame in processed_frames:
        # Encode the frame in JPEG format
        (flag, encoded_frame) = cv2.imencode(".jpg", frame)

        # Ensure the frame was successfully encoded
        if not flag:
            continue

        # Yield the output frame in the byte format
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
               bytearray(encoded_frame) + b'\r\n')

# Route for the video feed
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    # Start a new thread to process frames
    t = threading.Thread(target=process_frames)
    t.daemon = True
    t.start()

    # Run the Flask application
    print(f"To acces the Gloable link please click {public_url}")
    app.run(port=port_no)


[INFO] Loading YOLO from disk...
[INFO] Accessing video stream...
To acces the Gloable link please click https://4ff5-34-91-74-92.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:00:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:00:13] "GET /video_feed HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:00:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:00:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:00:19] "GET /video_feed HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:01:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:01:12] "GET /video_feed HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:01:19] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:01:19] "GET /video_feed HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2023 08:02:30] "GET / HTTP/1.1" 200 -
INFO: